### Step 1: Data Pre-Processing and Exploration

In [ ]:
# Create Annotation File from the dataset

from utils.create_dataset import create_annotation_file
fp = "./../data_folder/"
create_annotation_file(fp)


In [ ]:
# Create Test Train Split out of dataset
from utils.create_dataset import get_annotations_types, get_annotations
df = get_annotations('../dataset/annotations.csv')
types = get_annotations_types(df)

samples = []
for type in types:
    df = type.sample(n=2)
    samples.append(df)

print(samples)

In [15]:
import numpy as np
import pandas as pd
from utils.create_dataset import get_annotations_types, get_annotations
from sklearn.model_selection import train_test_split

df = get_annotations('../dataset/annotations.csv')
types = get_annotations_types(df)

df1 = types[0]

train, test = train_test_split(df1, random_state=15, train_size=0.7)

# print(train['filename'])
print(test['filename'].values)



['YAF_sell_angry.wav' 'YAF_week_angry.wav' 'YAF_merge_angry.wav'
 'YAF_read_angry.wav' 'YAF_thin_angry.wav' 'YAF_wag_angry.wav'
 'YAF_talk_angry.wav' 'YAF_rose_angry.wav' 'YAF_when_angry.wav'
 'YAF_pearl_angry.wav' 'YAF_tape_angry.wav' 'YAF_yes_angry.wav'
 'YAF_sub_angry.wav' 'YAF_reach_angry.wav' 'YAF_rag_angry.wav'
 'YAF_road_angry.wav' 'YAF_nice_angry.wav' 'YAF_soup_angry.wav'
 'YAF_south_angry.wav' 'YAF_mop_angry.wav' 'YAF_mess_angry.wav'
 'YAF_note_angry.wav' 'YAF_rough_angry.wav' 'YAF_peg_angry.wav'
 'YAF_raise_angry.wav' 'YAF_such_angry.wav' 'YAF_puff_angry.wav'
 'YAF_tip_angry.wav' 'YAF_yearn_angry.wav' 'YAF_neat_angry.wav']


In [1]:
from utils.create_dataset import split_data

split_data('../dataset/annotations.csv')


     type             filename                                    filepath
0   angry  YAF_shack_angry.wav  ./../data_folder/angry/YAF_shack_angry.wav
1   angry   YAF_pool_angry.wav   ./../data_folder/angry/YAF_pool_angry.wav
2   angry  YAF_shall_angry.wav  ./../data_folder/angry/YAF_shall_angry.wav
3   angry   YAF_page_angry.wav   ./../data_folder/angry/YAF_page_angry.wav
4   angry  YAF_perch_angry.wav  ./../data_folder/angry/YAF_perch_angry.wav
..    ...                  ...                                         ...
65  angry   YAF_size_angry.wav   ./../data_folder/angry/YAF_size_angry.wav
66  angry   YAF_wife_angry.wav   ./../data_folder/angry/YAF_wife_angry.wav
67  angry   YAF_moon_angry.wav   ./../data_folder/angry/YAF_moon_angry.wav
68  angry   YAF_ring_angry.wav   ./../data_folder/angry/YAF_ring_angry.wav
69  angry    YAF_nag_angry.wav    ./../data_folder/angry/YAF_nag_angry.wav

[70 rows x 3 columns]
    type            filename                                  filepath
0   fe

In [2]:
import os

angry = os.listdir("../dataset/train/angry/")
print(len(angry))

100
